# Recording a Gesture for Shimi

First, run the cell below to instatiate the shimi object. It should move to its initial positions and then relax.

**There's currently an issue where every other time the pypot motor library is run it crashes. Re-run the cell in this case.**

In [2]:
# Import some matplolib shortcuts for Jupyter notebook
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt

from shimi import *
import time
import datetime

from copy import deepcopy

import os

import threading

# Load Shimi model. Contains motor definitions and angle limits.
shimi = Shimi('config/shimi_robot_model.json')

# Allow the motors to be moved by hand.
shimi.make_compliant()

local_gestures = {}

Setting motors to starting positions {1: 10, 2: -21.599999999999994, 3: 0, 4: -11.85, 5: 5.41}


If you need to re-run the first cell, you must first close the connection to the motors by running this cell.

In [16]:
# Closes the serial connection to the motors
shimi.close()

The next cell gets a `recorder` object and starts a recording!

In the call to `get_recorder()`, you need to provide a list of motors you want to record from. The motors can be referenced as follows:
- `shimi.all_motors`
  - **This returns a list**, so no need to wrap this attribute when calling `get_recorder()`
- `shimi.torso`
- `shimi.neck_ud`
- `shimi.neck_lr`
- `shimi.phone`
- `shimi.foot`

In the call to `make_recording()`, edit the `wait_time` (time from when you run the cell to when it starts, in **integer** seconds) and the `recording_time` (in seconds, can be float) accordingly.

In [ ]:
# Get a recorder object for Shimi
recorder = shimi.get_recorder(shimi.all_motors)

# Make the recording
recorder = shimi.make_recording(recorder, wait_time=3, recording_time=10.0)

# Plot the recording
recorder.move.plot(plt.axes())

To play the recording you just made run the following cell.

In [ ]:
shimi.play_recordings([recorder.move])

If you are happy with that recording, hold onto it by running the following cell, **changing the name in the dictionary the recorder is being assigned to**! This is **only saving the gesture in memory in the python notebook** so if you want to save to disk, look at the next cell.

In [ ]:
# Change the string here or risk overwriting a previous gesture!!
name = 'myGesture'
local_gestures[name] = deepcopy(recorder.move)
print("Saved gesture " + name + " in this notebook.")

You can make sure it plays back by running the following cell.

In [ ]:
shimi.play_recordings([local_gestures[name]])

To save the recording to disk, run the following cell. It will save to 'recorded_gestures/name-timestamp.move`.

In [ ]:
ts = time.time()
timestamp = datetime.datetime.fromtimestamp(ts).strftime('%Y-%m-%d_%H:%M:%S')
foldername = '/home/nvidia/shimi/recorded_getsures'
filename = name + '-' + timestamp + '.move'

with open(os.path.join(foldername, filename), 'w') as f:
    local_gestures[name].save(f)
    
print("Saved gesture to " + os.path.join(foldername, filename))    

Here are some helper functions.

In [327]:
# Puts shimi in a neutral position with motors non-compliant
shimi.initial_position()

Setting motors to starting positions {1: 10, 2: -21.599999999999994, 3: 0, 4: -11.85, 5: 5.41}


## **Make sure Shimi will not collapse before you run this!!!**

In [ ]:
# Makes the motors compliant
shimi.make_compliant()

In [ ]:
for m in shimi.all_motors:
    print("speed: {0} | goal_pos: {1} | pres_pos: {2}".format(m.moving_speed, m.goal_position, m.present_position))

In [321]:
min_vel = 20
max_vel = 100 # degrees per second

def move_with_acceleration(motors, positions, duration, update_freq):
    start = time.time()
    
    while time.time() <= start + duration:
        for i, m in enumerate(motors):
            rel_pos = abs(duration / 2 - (time.time() - start))
            vel = max_vel * (2 * (1.0 - rel_pos / duration) - 1)
            m.goal_position = positions[i]
            m.moving_speed = vel
            print("goal: {0}, pres: {1}".format(vel, m.present_speed))

        time.sleep(update_freq)
    
def move_with_acceleration_io(dxl_io, motors, positions, duration, update_freq):
    start = time.time()
    starting_positions = dxl_io.get_present_position(motors)
    starting_positions = np.array(starting_positions)
    positions = np.array(positions)
    
    top_speeds = 2 * ((abs((positions - starting_positions) / 2) - (min_vel * (duration / 2))) / ((duration / 2) ** 2))
    print("top speeds", top_speeds)
    
    dxl_io.set_moving_speed(dict(zip(motors, [min_vel for _ in range(len(motors))])))
    dxl_io.set_goal_position(dict(zip(motors, positions)))

    time.sleep(update_freq)
        
    while time.time() <= start + duration:
        rel_pos = abs(duration / 2 - (time.time() - start))
        vel = (top_speeds * (2 * (1.0 - rel_pos / duration) - 1)) + min_vel
#         print(vel)
        dxl_io.set_moving_speed(dict(zip(motors, vel)))
#         print("goal: {0}, pres: {1}".format(vel, dxl_io.get_moving_speed(motors)))
            
        time.sleep(update_freq)
    
    print("done. goal pos: {0} | pres pos: {1}".format(dxl_io.get_goal_position(motors), dxl_io.get_present_position(motors)))

In [13]:
move_with_acceleration([shimi.neck_ud], [], 3.0, 0.2)

goal: 2.1457672119140625e-05, pres: 0.0
goal: 2.050459384918213, pres: 0.0
goal: 4.075860977172852, pres: 213.408
goal: 6.097710132598877, pres: 0.0
goal: 8.154571056365967, pres: 0.0
goal: 9.795689582824707, pres: 0.0
goal: 7.781350612640381, pres: 0.0
goal: 5.768928527832031, pres: 0.0
goal: 3.757469654083252, pres: 0.0
goal: 1.7447495460510254, pres: 0.0


In [17]:
ports = pypot.dynamixel.get_available_ports()
dxl_io = pypot.dynamixel.DxlIO(ports[0])

ids = dxl_io.scan(range(10))
print('found:', ids)

found: [1, 2, 3, 4, 5]


In [329]:
move_with_acceleration_io(dxl_io, [2], [-50], 2.0, 0.05)

top speeds [21.16]
done. goal pos: (-49.98,) | pres pos: (-52.44,)


In [347]:
t1_a = threading.Thread(target=lambda: move_with_acceleration_io(dxl_io, [2], [60], 2.0, 0.05))
t1_b = threading.Thread(target=lambda: move_with_acceleration_io(dxl_io, [3], [-50], 2.0, 0.05))
t1_a.start()
t1_b.start()
t1_a.join()
t1_b.join()
print("next motions")
t1_a = threading.Thread(target=lambda: move_with_acceleration_io(dxl_io, [2], [-100], 2.4, 0.05))
t1_b = threading.Thread(target=lambda: move_with_acceleration_io(dxl_io, [3], [0], 3.0, 0.05))
t1_a.start()
t1_b.start()
t1_a.join()
t1_b.join()
print("done")

top speeds [119.21]
top speeds [10.84]
done. goal pos: (60.0,) | pres pos: (58.51,)
done. goal pos: (-49.98,) | pres pos: (-48.84,)
next motions
top speeds [77.04861111]
top speeds [-4.92444444]
done. goal pos: (-100.0,) | pres pos: (-80.31,)
done. goal pos: (0.04,) | pres pos: (0.4,)
done
